In [28]:
import glob

import xarray as xr
import numpy as np

In [29]:
infiles = glob.glob('/g/data/xv83/ds0092/CAFE/forecasts/f6/WIP/c5-d60-pX-f6-19??????/ZARR/atmos_isobaric_daily.zarr.zip')
infiles.sort()

In [30]:
infiles

['/g/data/xv83/ds0092/CAFE/forecasts/f6/WIP/c5-d60-pX-f6-19861101/ZARR/atmos_isobaric_daily.zarr.zip',
 '/g/data/xv83/ds0092/CAFE/forecasts/f6/WIP/c5-d60-pX-f6-19871101/ZARR/atmos_isobaric_daily.zarr.zip',
 '/g/data/xv83/ds0092/CAFE/forecasts/f6/WIP/c5-d60-pX-f6-19891101/ZARR/atmos_isobaric_daily.zarr.zip',
 '/g/data/xv83/ds0092/CAFE/forecasts/f6/WIP/c5-d60-pX-f6-19901101/ZARR/atmos_isobaric_daily.zarr.zip',
 '/g/data/xv83/ds0092/CAFE/forecasts/f6/WIP/c5-d60-pX-f6-19911101/ZARR/atmos_isobaric_daily.zarr.zip',
 '/g/data/xv83/ds0092/CAFE/forecasts/f6/WIP/c5-d60-pX-f6-19921101/ZARR/atmos_isobaric_daily.zarr.zip',
 '/g/data/xv83/ds0092/CAFE/forecasts/f6/WIP/c5-d60-pX-f6-19931101/ZARR/atmos_isobaric_daily.zarr.zip']

## Open and clip a single file

In [31]:
ds = xr.open_zarr(infiles[0], consolidated=True, use_cftime=True)

In [32]:
ds

<xarray.Dataset>
Dimensions:         (ensemble: 96, lat: 90, latb: 91, level: 21, lon: 144, lonb: 145, nv: 2, time: 3653)
Coordinates: (12/14)
    area            (lat, lon) float32 dask.array<chunksize=(90, 144), meta=np.ndarray>
    average_DT      (time) timedelta64[ns] dask.array<chunksize=(3653,), meta=np.ndarray>
    average_T1      (time) object dask.array<chunksize=(3653,), meta=np.ndarray>
    average_T2      (time) object dask.array<chunksize=(3653,), meta=np.ndarray>
  * ensemble        (ensemble) int64 1 2 3 4 5 6 7 8 ... 89 90 91 92 93 94 95 96
  * lat             (lat) float64 -89.49 -87.98 -85.96 ... 85.96 87.98 89.49
    ...              ...
  * lon             (lon) float64 1.25 3.75 6.25 8.75 ... 353.8 356.2 358.8
  * lonb            (lonb) float64 0.0 2.5 5.0 7.5 ... 352.5 355.0 357.5 360.0
  * nv              (nv) float64 1.0 2.0
  * time            (time) object 1986-11-01 12:00:00 ... 1996-10-31 12:00:00
    time_bounds     (time, nv) timedelta64[ns] dask.array<chunksize=(3653, 2), meta=np.ndarray>
    zsurf           (lat, lon) float32 dask.array<chunksize=(90, 144), meta=np.ndarray>
Data variables: (12/38)
    DELP            (time, ensemble, level, lat, lon) float32 dask.array<chunksize=(28, 96, 1, 90, 144), meta=np.ndarray>
    awp             (time, ensemble, lat, lon) float32 dask.array<chunksize=(28, 96, 90, 144), meta=np.ndarray>
    evap            (time, ensemble, lat, lon) float32 dask.array<chunksize=(28, 96, 90, 144), meta=np.ndarray>
    h500            (time, ensemble, lat, lon) float32 dask.array<chunksize=(28, 96, 90, 144), meta=np.ndarray>
    hght            (time, ensemble, level, lat, lon) float32 dask.array<chunksize=(28, 96, 1, 90, 144), meta=np.ndarray>
    lwdn_sfc        (time, ensemble, lat, lon) float32 dask.array<chunksize=(28, 96, 90, 144), meta=np.ndarray>
    ...              ...
    u_ref           (time, ensemble, lat, lon) float32 dask.array<chunksize=(28, 96, 90, 144), meta=np.ndarray>
    ucomp           (time, ensemble, level, lat, lon) float32 dask.array<chunksize=(28, 96, 1, 90, 144), meta=np.ndarray>
    v_ref           (time, ensemble, lat, lon) float32 dask.array<chunksize=(28, 96, 90, 144), meta=np.ndarray>
    vcomp           (time, ensemble, level, lat, lon) float32 dask.array<chunksize=(28, 96, 1, 90, 144), meta=np.ndarray>
    vis_exopd_vl_c  (time, ensemble, lat, lon) float32 dask.array<chunksize=(28, 96, 90, 144), meta=np.ndarray>
    wvp             (time, ensemble, lat, lon) float32 dask.array<chunksize=(28, 96, 90, 144), meta=np.ndarray>
Attributes:
    comment:    pressure level interpolator, version 3.0, precision=double
    filename:   atmos_isobaric_daily.zarr
    grid_tile:  N/A
    grid_type:  regular
    title:      AccessOcean-AM2

In [33]:
da = ds['precip']
for drop_coord in ['average_DT', 'average_T1', 'average_T2', 'zsurf', 'area']:
    if drop_coord in da.coords:
        da = da.drop(drop_coord)

In [34]:
da

,Array,Chunk
Bytes,18.18 GB,139.35 MB
Shape,"(3653, 96, 90, 144)","(28, 96, 90, 144)"
Count,132 Tasks,131 Chunks
Type,float32,numpy.ndarray


In [35]:
da['time']

<xarray.DataArray 'time' (time: 3653)>
array([cftime.DatetimeJulian(1986, 11, 1, 12, 0, 0, 0),
       cftime.DatetimeJulian(1986, 11, 2, 12, 0, 0, 0),
       cftime.DatetimeJulian(1986, 11, 3, 12, 0, 0, 0), ...,
       cftime.DatetimeJulian(1996, 10, 29, 12, 0, 0, 0),
       cftime.DatetimeJulian(1996, 10, 30, 12, 0, 0, 0),
       cftime.DatetimeJulian(1996, 10, 31, 12, 0, 0, 0)], dtype=object)
Coordinates:
  * time     (time) object 1986-11-01 12:00:00 ... 1996-10-31 12:00:00
Attributes:
    bounds:          time_bounds
    calendar_type:   JULIAN
    cartesian_axis:  T
    long_name:       time

In [36]:
#da['time'] = da.indexes['time'].to_datetimeindex()

In [37]:
#da['time']

In [38]:
da['lat']

<xarray.DataArray 'lat' (lat: 90)>
array([-89.494382, -87.977528, -85.955056, -83.932584, -81.910112, -79.88764 ,
       -77.865169, -75.842697, -73.820225, -71.797753, -69.775281, -67.752809,
       -65.730337, -63.707865, -61.685393, -59.662921, -57.640449, -55.617978,
       -53.595506, -51.573034, -49.550562, -47.52809 , -45.505618, -43.483146,
       -41.460674, -39.438202, -37.41573 , -35.393258, -33.370787, -31.348315,
       -29.325843, -27.303371, -25.280899, -23.258427, -21.235955, -19.213483,
       -17.191011, -15.168539, -13.146067, -11.123596,  -9.101124,  -7.078652,
        -5.05618 ,  -3.033708,  -1.011236,   1.011236,   3.033708,   5.05618 ,
         7.078652,   9.101124,  11.123596,  13.146067,  15.168539,  17.191011,
        19.213483,  21.235955,  23.258427,  25.280899,  27.303371,  29.325843,
        31.348315,  33.370787,  35.393258,  37.41573 ,  39.438202,  41.460674,
        43.483146,  45.505618,  47.52809 ,  49.550562,  51.573034,  53.595506,
        55.617978,  57.640449,  59.662921,  61.685393,  63.707865,  65.730337,
        67.752809,  69.775281,  71.797753,  73.820225,  75.842697,  77.865169,
        79.88764 ,  81.910112,  83.932584,  85.955056,  87.977528,  89.494382])
Coordinates:
  * lat      (lat) float64 -89.49 -87.98 -85.96 -83.93 ... 85.96 87.98 89.49
Attributes:
    cartesian_axis:  Y
    edges:           latb
    long_name:       latitude
    units:           degrees_N

In [39]:
da['lon']

<xarray.DataArray 'lon' (lon: 144)>
array([  1.25,   3.75,   6.25,   8.75,  11.25,  13.75,  16.25,  18.75,  21.25,
        23.75,  26.25,  28.75,  31.25,  33.75,  36.25,  38.75,  41.25,  43.75,
        46.25,  48.75,  51.25,  53.75,  56.25,  58.75,  61.25,  63.75,  66.25,
        68.75,  71.25,  73.75,  76.25,  78.75,  81.25,  83.75,  86.25,  88.75,
        91.25,  93.75,  96.25,  98.75, 101.25, 103.75, 106.25, 108.75, 111.25,
       113.75, 116.25, 118.75, 121.25, 123.75, 126.25, 128.75, 131.25, 133.75,
       136.25, 138.75, 141.25, 143.75, 146.25, 148.75, 151.25, 153.75, 156.25,
       158.75, 161.25, 163.75, 166.25, 168.75, 171.25, 173.75, 176.25, 178.75,
       181.25, 183.75, 186.25, 188.75, 191.25, 193.75, 196.25, 198.75, 201.25,
       203.75, 206.25, 208.75, 211.25, 213.75, 216.25, 218.75, 221.25, 223.75,
       226.25, 228.75, 231.25, 233.75, 236.25, 238.75, 241.25, 243.75, 246.25,
       248.75, 251.25, 253.75, 256.25, 258.75, 261.25, 263.75, 266.25, 268.75,
       271.25, 273.75, 276.25, 278.75, 281.25, 283.75, 286.25, 288.75, 291.25,
       293.75, 296.25, 298.75, 301.25, 303.75, 306.25, 308.75, 311.25, 313.75,
       316.25, 318.75, 321.25, 323.75, 326.25, 328.75, 331.25, 333.75, 336.25,
       338.75, 341.25, 343.75, 346.25, 348.75, 351.25, 353.75, 356.25, 358.75])
Coordinates:
  * lon      (lon) float64 1.25 3.75 6.25 8.75 11.25 ... 351.2 353.8 356.2 358.8
Attributes:
    cartesian_axis:  X
    edges:           lonb
    long_name:       longitude
    units:           degrees_E

In [40]:
def get_region(da, box):
    """Select grid points that fall within a lat/lon box.
    
    Args:
      da (xarray DataArray)
      box (array-like) : [south bound, north bound, east bound, west bound]
    
    """

    lat_south_bound, lat_north_bound, lon_east_bound, lon_west_bound = box
    assert -90 <= lat_south_bound <= 90, "Valid latitude range is [-90, 90]"
    assert -90 <= lat_north_bound <= 90, "Valid latitude range is [-90, 90]"
    assert lat_south_bound < lat_north_bound, "South bound greater than north bound"
    assert 0 <= lon_east_bound < 360, "Valid longitude range is [0, 360)"
    assert 0 <= lon_west_bound < 360, "Valid longitude range is [0, 360)"
    
    da = da.assign_coords({'lon': (da['lon'] + 360)  % 360})
        
    mask_lat = (da['lat'] > lat_south_bound) & (da['lat'] < lat_north_bound)
    if lon_east_bound < lon_west_bound:
        mask_lon = (da['lon'] > lon_east_bound) & (da['lon'] < lon_west_bound)
    else:
        mask_lon = (da['lon'] > lon_east_bound) | (da['lon'] < lon_west_bound)
    
    da = da.where(mask_lat & mask_lon, drop=True) 
        
    #if sort:
    #    da = da.sortby(lat_name).sortby(lon_name)
    # da.sel({lat_name: slice(box[0], box[1]), lon_name: slice(box[2], box[3])})

    return da 

In [41]:
da = get_region(da, [-46, -9, 111, 157])

/g/data/e14/dbi599/miniconda3/envs/unseen/lib/python3.8/site-packages/xarray/core/indexing.py:1375: PerformanceWarning: Slicing is producing a large chunk. To accept the large
chunk and silence this warning, set the option
    >>> with dask.config.set(**{'array.slicing.split_large_chunks': False}):
    ...     array[indexer]

To avoid creating the large chunks, set the option
    >>> with dask.config.set(**{'array.slicing.split_large_chunks': True}):
    ...     array[indexer]
  value = value[(slice(None),) * axis + (subkey,)]


In [42]:
da

,Array,Chunk
Bytes,506.39 MB,3.88 MB
Shape,"(3653, 96, 19, 19)","(28, 96, 19, 19)"
Count,526 Tasks,131 Chunks
Type,float32,numpy.ndarray


In [43]:
da['lon']

<xarray.DataArray 'lon' (lon: 19)>
array([111.25, 113.75, 116.25, 118.75, 121.25, 123.75, 126.25, 128.75, 131.25,
       133.75, 136.25, 138.75, 141.25, 143.75, 146.25, 148.75, 151.25, 153.75,
       156.25])
Coordinates:
  * lon      (lon) float64 111.2 113.8 116.2 118.8 ... 148.8 151.2 153.8 156.2

In [44]:
da['lat']

<xarray.DataArray 'lat' (lat: 19)>
array([-45.505618, -43.483146, -41.460674, -39.438202, -37.41573 , -35.393258,
       -33.370787, -31.348315, -29.325843, -27.303371, -25.280899, -23.258427,
       -21.235955, -19.213483, -17.191011, -15.168539, -13.146067, -11.123596,
        -9.101124])
Coordinates:
  * lat      (lat) float64 -45.51 -43.48 -41.46 -39.44 ... -13.15 -11.12 -9.101
Attributes:
    cartesian_axis:  Y
    edges:           latb
    long_name:       latitude
    units:           degrees_N

## Open and clip a bunch of files

In [45]:
def open_and_clip(infile):
    """Open file and select region"""
    
    ds = xr.open_zarr(infile, consolidated=True, use_cftime=True)
    
    da = ds['precip']
    for drop_coord in ['average_DT', 'average_T1', 'average_T2', 'zsurf', 'area']:
        if drop_coord in da.coords:
            da = da.drop(drop_coord)
    
    da = get_region(da, [-46, -9, 111, 157])
    
    return da

In [46]:
datasets = []
for infile in infiles[0:2]:
    da = open_and_clip(infile)
    lead_time = range(len(da['time']))
    init_date = np.datetime64(da['time'].values[0].strftime('%Y-%m-%d'))            
    datasets.append(da.rename({'time': 'lead_time'}).assign_coords({'lead_time': lead_time, 'init_date': init_date}))
da = xr.concat(datasets, dim='init_date')

/g/data/e14/dbi599/miniconda3/envs/unseen/lib/python3.8/site-packages/xarray/core/indexing.py:1375: PerformanceWarning: Slicing is producing a large chunk. To accept the large
chunk and silence this warning, set the option
    >>> with dask.config.set(**{'array.slicing.split_large_chunks': False}):
    ...     array[indexer]

To avoid creating the large chunks, set the option
    >>> with dask.config.set(**{'array.slicing.split_large_chunks': True}):
    ...     array[indexer]
  value = value[(slice(None),) * axis + (subkey,)]
/g/data/e14/dbi599/miniconda3/envs/unseen/lib/python3.8/site-packages/xarray/core/indexing.py:1375: PerformanceWarning: Slicing is producing a large chunk. To accept the large
chunk and silence this warning, set the option
    >>> with dask.config.set(**{'array.slicing.split_large_chunks': False}):
    ...     array[indexer]

To avoid creating the large chunks, set the option
    >>> with dask.config.set(**{'array.slicing.split_large_chunks': True}):
    ...     

In [47]:
da

,Array,Chunk
Bytes,1.01 GB,3.88 MB
Shape,"(2, 3653, 96, 19, 19)","(1, 28, 96, 19, 19)"
Count,1575 Tasks,262 Chunks
Type,float32,numpy.ndarray


In [48]:
time_values = [da.get_index('init_date').shift(int(lead), 'D') for lead in da['lead_time']]
target_times = xr.DataArray(time_values,
                            dims={'lead_time': da['lead_time'],
                                  'init_date': da['init_date']})
da['lead_time'].attrs['units'] = 'D'
da = da.assign_coords({'time': target_times})

In [49]:
da

,Array,Chunk
Bytes,1.01 GB,3.88 MB
Shape,"(2, 3653, 96, 19, 19)","(1, 28, 96, 19, 19)"
Count,1575 Tasks,262 Chunks
Type,float32,numpy.ndarray


In [50]:
da['time'][0:5, 0]

<xarray.DataArray 'time' (lead_time: 5)>
array(['1986-11-01T00:00:00.000000000', '1986-11-02T00:00:00.000000000',
       '1986-11-03T00:00:00.000000000', '1986-11-04T00:00:00.000000000',
       '1986-11-05T00:00:00.000000000'], dtype='datetime64[ns]')
Coordinates:
  * lead_time  (lead_time) int64 0 1 2 3 4
    init_date  datetime64[ns] 1986-11-01
    time       (lead_time) datetime64[ns] 1986-11-01 1986-11-02 ... 1986-11-05

In [51]:
da['time'][0:5, 1]

<xarray.DataArray 'time' (lead_time: 5)>
array(['1987-11-01T00:00:00.000000000', '1987-11-02T00:00:00.000000000',
       '1987-11-03T00:00:00.000000000', '1987-11-04T00:00:00.000000000',
       '1987-11-05T00:00:00.000000000'], dtype='datetime64[ns]')
Coordinates:
  * lead_time  (lead_time) int64 0 1 2 3 4
    init_date  datetime64[ns] 1987-11-01
    time       (lead_time) datetime64[ns] 1987-11-01 1987-11-02 ... 1987-11-05

In [52]:
da['time'][0, :]

<xarray.DataArray 'time' (init_date: 2)>
array(['1986-11-01T00:00:00.000000000', '1987-11-01T00:00:00.000000000'],
      dtype='datetime64[ns]')
Coordinates:
    lead_time  int64 0
  * init_date  (init_date) datetime64[ns] 1986-11-01 1987-11-01
    time       (init_date) datetime64[ns] 1986-11-01 1987-11-01

In [55]:
outfile = '/g/data/xv83/dbi599/precip_c5-d60-pX-f6_19861101-19871101_atmos_isobaric_daily.zarr.zip'
ds = da.to_dataset()
ds.to_zarr(outfile, mode='w')